In [1]:
import os
import subprocess
import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
city="ams"
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
print(base_dir)
# Paths for the Population Data --------------------------------------------------------------
#path to ancillary data folder
ancillary_data_folder_path = base_dir + "/data_prep/{}_ProjectData/AncillaryData".format(city)
ancillary_POPdata_folder_path = base_dir + "/data_prep/{}_ProjectData/PopData".format(city)
image_path= base_dir + "/data_prep/{}_Projectdata/PopData/images".format(city)
gdal_rasterize_path = r'C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin'
years_list=[2002,2004,2006,2008,2010,2012,2014,2016,2018] 


c:\FUME\PopNetV2


In [3]:
## ## ## ## ## ----- CREATE NEW FOLDER  ----- ## ## ## ## ##
def createFolder(path):
    if not os.path.exists(path):
        print("------------------------------ Creating Folder : {} ------------------------------".format(path))
        os.makedirs(path)
    else: 
        print("------------------------------ Folder already exists------------------------------")

In [4]:
def calcMeanHomeValues(outPath, year):
    srcPath = ancillary_data_folder_path + "/home_value/"
    for file in os.listdir(srcPath):
        if file.endswith('.shp') and file.startswith('WONINGWAARDE_{0}_'.format(year)) :#
            path = Path(srcPath + "/{0}".format(file))
            polydf = gpd.read_file(path, crs="EPSG:28992")
            polydf = polydf.to_crs("EPSG:3035")

            grid_df = gpd.read_file(ancillary_POPdata_folder_path + '/{0}/temp_shp/{0}_dataVectorGrid.geojson'.format(year))
            grid_df.head(2)

            join_left_df = sjoin(grid_df, polydf, how="left")
            join_left_df.head(2)
            frame = join_left_df.drop_duplicates(subset=['grid_id'], keep='first')
            col = frame.loc[: , "Lower":"Upper"]
            frame['price_mean'] = col.mean(axis=1)
            
            ndf= frame[['grid_id','geometry','Lower','Upper','price_mean']]
            ndf.to_file(outPath + "{}_residenceValue.geojson".format(year) ,driver='GeoJSON',crs="EPSG:3035")

In [9]:
# Normalise the prices based on max in 2002
def normMeanHomeValues(outPath, destPath, year):
    print(outPath)
    for file in os.listdir(outPath):
        print(file)
        if  file.startswith('{0}_'.format(year)) :
            path = Path(outPath + "/{0}".format(file))
            frame = gpd.read_file(path, crs="EPSG:3035")
            
            #Read file for base of Normalisation (2002) and get max value
            maxValuePath = outPath + '/2002_residenceValue.geojson'
            maxValueFile = gpd.read_file(maxValuePath, crs="EPSG:3035")
            maxValue = maxValueFile['price_mean'].max()
            
            # Normalise
            frame['price_meanNorm'] = frame['price_mean'] / maxValueFile['price_mean']
            ndf = frame[['grid_id','geometry','price_meanNorm']]
            
            # Save file
            ndf.to_file(destPath + "{}_residenceValueNorm.geojson".format(year) ,driver='GeoJSON',crs="EPSG:3035")

In [7]:
def shptoraster(srcPath, dstPath, ancillary_data_folder_path, gdal_rasterize_path, city,year):
    # Getting extent of ghs pop raster
    data = gdal.Open(os.path.dirname(ancillary_data_folder_path) + "/temp_tif/{0}_CLC_2012_2018.tif".format(city))
    wkt = data.GetProjection()
    geoTransform = data.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1] * data.RasterXSize
    miny = maxy + geoTransform[5] * data.RasterYSize
    data = None
    xres=100
    yres=100
    # Rasterizing layers
    src_file = srcPath + "{}_residenceValue.geojson".format(year)
    df = gpd.read_file(src_file)
    
    column_name = 'price_mean'
    print("Rasterizing {} layer".format(column_name))
    dst_file = dstPath + "/{0}_{1}.tif".format(year, column_name)
    cmd = '{0}/gdal_rasterize.exe -a {9} -te {1} {2} {3} {4} -tr {5} {6} "{7}" "{8}"'\
        .format(gdal_rasterize_path, minx, miny, maxx, maxy, xres, yres, src_file, dst_file, column_name)
    print(cmd) #-ot Integer64
    subprocess.call(cmd, shell=True)

In [9]:
outPath = os.path.dirname(ancillary_data_folder_path) + "/temp_shp/home_prices/"
#destPath  = os.path.dirname(ancillary_data_folder_path) + "/Normalized/02HomePrices_Norm/"
destPath  = os.path.dirname(ancillary_data_folder_path) + "/temp_tif/ams_home_prices/"
#createFolder(outPath)
createFolder(destPath)

for year in years_list:
    #calcMeanHomeValues(outPath, year)
    #normMeanHomeValues(outPath,destPath, year)
    shptoraster(outPath, destPath, ancillary_data_folder_path, gdal_rasterize_path, city,year)

------------------------------ Folder already exists------------------------------
Rasterizing price_mean layer
C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin/gdal_rasterize.exe -a price_mean -te 3962600.0 3252900.0 3986200.0 3270800.0 -tr 100 100 "c:\FUME\PopNetV2/data_prep/ams_ProjectData/temp_shp/home_prices/2002_residenceValue.geojson" "c:\FUME\PopNetV2/data_prep/ams_ProjectData/temp_tif/ams_home_prices//2002_price_mean.tif"
Rasterizing price_mean layer
C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin/gdal_rasterize.exe -a price_mean -te 3962600.0 3252900.0 3986200.0 3270800.0 -tr 100 100 "c:\FUME\PopNetV2/data_prep/ams_ProjectData/temp_shp/home_prices/2004_residenceValue.geojson" "c:\FUME\PopNetV2/data_prep/ams_ProjectData/temp_tif/ams_home_prices//2004_price_mean.tif"
Rasterizing price_mean layer
C:/Users/NM12LQ/Anaconda3/envs/popnet_env/Library/bin/gdal_rasterize.exe -a price_mean -te 3962600.0 3252900.0 3986200.0 3270800.0 -tr 100 100 "c:\FUME\PopNetV2/data_prep/ams